In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 599.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 19.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving Movement Summary_JN-1_Site_Arrival & Departure Event-Amanah Final Proposal-GHR (2).pdf to Movement Summary_JN-1_Site_Arrival & Departure Event-Amanah Final Proposal-GHR (2).pdf
Saving Movement Summary_JN-1_Site_Arrival & Departure Event-Amanah Final Proposal-GHR.pdf to Movement Summary_JN-1_Site_Arrival & Departure Event-Amanah Final Proposal-GHR.pdf
Saving Movement Summary_Jn-2_01 U-Turn_Site_Arrival & Departure Event-Amanah Final Proposal-GHR (2).pdf to Movement Summary_Jn-2_01 U-Turn_Site_Arrival & Departure Event-Amanah Final Proposal-GHR (2).pdf
Saving Movement Summary_Jn-2_01 U-Turn_Site_Arrival & Departure Event-Amanah Final Proposal-GHR.pdf to Movement Summary_Jn-2_01 U-Turn_Site_Arrival & Departure Event-Amanah Final Proposal-GHR.pdf
Saving Movement Summary_Jn-2_02 U-Turn_Site_Arrival & Departure Event-Amanah Final Proposal-GHR (2).pdf to Movement Summary_Jn-2_02 U-Turn_Site_Arrival & Departure Event-Amanah Final Proposal-GHR (2).pdf
Saving Movement Summary_Jn-2_02 U-Tu

In [ ]:
def categorize_value(B17):
    if B17 <= 10:
        return "A"
    elif 10 < B17 <= 15:
        return "B"
    elif 15 < B17 <= 25:
        return "C"
    elif 25 < B17 <= 35:
        return "D"
    elif 35 < B17 <= 50:
        return "E"
    else:
        return "F"

In [ ]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re
Output = pd.DataFrame(columns= ['Scenario','Location', 'Delay', 'LOS', 'Max_Queue_Length', 'Approach'] )

In [ ]:
for filename in uploaded.keys():
    all_text = []
    with pdfplumber.open(filename) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:  # Ensure the page contains text
                all_text.append(f"Page {page_num}:\n{page_text}\n")

    # Join the text of all pages into a single string, separating by newlines
    full_text = "\n".join(all_text)
    data = []
    for line in full_text.split('\n'):
        data.append(line)
    pattern = r"Site: (\S+)\s\[(.*?)\s\(Site Folder: (.*?)\)\]"
    # Search the text using the regular expression
    match = re.search(pattern, data[2])
    if match:
        site_number = match.group(1)
        tmc_number = match.group(2)
        scenario = match.group(3)
    start_index = None
    for i, line in enumerate(data):
        if 'All Vehicles' in line:
            start_index = i
            break
    filtered_data = data[:start_index+1]
    split_data = [row.split() for row in filtered_data[12:]]
    df= pd.DataFrame(split_data)
    Delay = df.iloc[-1,7]
    LOS = categorize_value(float(Delay))
    def assign_direction(df):
        direction = None  # Initialize direction as None
        directions = []  # List to store the directions for each row
        for i, row in df.iterrows():
            if isinstance(row[0], str):  # Ensure the value in the first column is a string
        # Initialize a temporary direction variable
                temp_direction = ""

        # Check for all possible directions
                if 'South' in row[0]:
                    temp_direction += "South"
                if 'East' in row[0]:
                    temp_direction += "East"
                if 'North' in row[0]:
                    temp_direction += "North"
                if 'West' in row[0]:
                    temp_direction += "West"

        # If 'Approach' is in the row, retain the last valid direction
                if 'Approach' in row[0]:
                    temp_direction = direction  # Retain the previous direction if 'Approach' is encountered

        # Update the main direction variable if a valid direction is found
                if temp_direction:
                    direction = temp_direction

            directions.append(direction)  # Append the determined direction to the list
        # Add the direction as a new column
        df['Approach Direction'] = directions
        return df

    # Apply the function to assign direction to all rows
    df = assign_direction(df)
    Que = df[df[0].apply(lambda x: len(str(x)) <= 3)].iloc[:-1,:]
    Que = Que.reset_index(drop=True)
    Que.iloc[:, 13] = pd.to_numeric(Que.iloc[:, 13], errors='coerce')
    index_of_max = Que.iloc[:, 13].idxmax()
    Approch = Que.iloc[index_of_max, -1]
    Max_Queue_Length = Que.iloc[index_of_max, 13]
    new_row = {
        'Scenario': scenario,
        'Location': tmc_number,
        'Delay': Delay,
        'LOS': LOS,
        'Max_Queue_Length': Max_Queue_Length,
        'Approach': Approch
    }
    # Use pd.concat to add the new row
    Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)

<ipython-input-5-889373088f3c>:79: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)


In [ ]:
Output.to_excel('Summary_Report.xlsx', index=False)